In [8]:
import os
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

tfidf_vectorizer = TfidfVectorizer()

file_list = []
file_list.append(r'./200416_label0_dataset_sentence.xlsx')
file_list.append(r'./200416_label1_dataset_sentence.xlsx')
file_list.append(r'./200416_label2_dataset_sentence.xlsx')
file_list.append(r'./200416_label3_dataset_sentence.xlsx')
file_list.append(r'./200416_label4_dataset_sentence.xlsx')
file_list.append(r'./200416_label5_dataset_sentence.xlsx')
file_list.append(r'./200416_label6_dataset_sentence.xlsx')

In [9]:

def baseline_model(texts,labels,Round,sender_num):
    
    x_train = tfidf_vectorizer.fit_transform(texts).toarray()
#     x_test = tfidf_vectorizer.transform(x_test)
    svd = TruncatedSVD(n_components=400).fit(x_train)
    reduced = svd.transform(x_train)
    total = reduced.tolist()
    for item in range(len(total)):
        total[item] = np.append(total[item], sender_num[item])
    for item in range(len(total)):
        total[item] = np.append(total[item], Round[item])
    
    Model_name = ['XGBClassifier','BernoulliNB','Svm (linear)','Logistic Regression',
              'Random Forest','kNN','Decision Tree']
    function = [XGBClassifier(),BernoulliNB(),svm.SVC(kernel="linear"),LogisticRegression(),
              RandomForestClassifier(),KNeighborsClassifier(),DecisionTreeClassifier()]
    #Model_name = ['BernoulliNB']
    #function = [BernoulliNB()]
        
    
    perform_f1 = []
    perform_acc = []
    perform_recall = []
    perform_precision = []
    perform_data_f1 = []
    perform_data_acc = []
    
    all_df = pd.DataFrame({'file_name':[''],
                           'model':[''],
                           'precision':[''],
                           'recall':[''],
                           'f1-score':[''],
                           'support':[''] })


        
    for i in tqdm(range(len(function))):

        if Model_name[i] == 'XGBClassifier':
            new_total = np.array(total)
        else:
            new_total = total                


        kf = KFold(n_splits=10, shuffle=True)       
        model = function[i]
        
        predicted = []
        expected = []
        for train_index, test_index in kf.split(new_total):
            x_train = np.array(new_total)[train_index]
            y_train = np.array(labels)[train_index]
            x_test = np.array(new_total)[test_index]
            y_test = np.array(labels)[test_index]

            model.fit(x_train, y_train)
            expected.extend(y_test)
            predicted.extend(model.predict(x_test))
        
        print('Model Name:'+ Model_name[i])
        print(metrics.classification_report(expected, predicted))
        print('weighted-avg prf: {0}, {1}, {2}'.format(
        metrics.precision_score(expected, predicted, average='weighted'),
        metrics.recall_score(expected, predicted, average='weighted'),
        metrics.f1_score(expected, predicted, average='weighted'))
        )       

  
        report = metrics.classification_report(expected, predicted, output_dict=True)
        df = pd.DataFrame(report).transpose()
        df.insert(0,'model',Model_name[i])
        
        all_df = all_df.append(df)
        
       
    return all_df



In [10]:

for x in range(len(file_list)):
    dataset = pd.read_excel(file_list[x])

    tmp = []
    for i in dataset['sender']:
        if i == 'customer':
            tmp.append(0)
        else:
            tmp.append(1)
    dataset['sender_num'] = tmp
    dataset.head()

    texts = list(dataset["texts"])
    labels = list(dataset["labels"])
    Round = list(dataset["round"])
    sender_num = list(dataset["sender_num"])
    
    #baseline_model(texts,labels,Round,sender_num)
    df_report = baseline_model(texts,labels,Round,sender_num)
    df_report.to_csv('./Kfold'+ str(x) +'.csv')
    
    
    

Model Name:XGBClassifier
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     10668
           1       1.00      0.83      0.91      6487

    accuracy                           0.94     17155
   macro avg       0.95      0.92      0.93     17155
weighted avg       0.94      0.94      0.93     17155

weighted-avg prf: 0.9413047629035085, 0.9356455843777325, 0.9342804543457371


C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


Model Name:BernoulliNB
              precision    recall  f1-score   support

           0       0.85      0.80      0.83     10668
           1       0.70      0.77      0.74      6487

    accuracy                           0.79     17155
   macro avg       0.78      0.79      0.78     17155
weighted avg       0.80      0.79      0.79     17155

weighted-avg prf: 0.7965254821286073, 0.7910230253570387, 0.7926890832633282


C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


Model Name:Svm (linear)
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     10668
           1       1.00      0.83      0.91      6487

    accuracy                           0.94     17155
   macro avg       0.95      0.91      0.93     17155
weighted avg       0.94      0.94      0.93     17155

weighted-avg prf: 0.9416535642834423, 0.9356455843777325, 0.9342391077339562


...

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


Model Name:kNN
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9478
           1       1.00      1.00      1.00      7677

    accuracy                           1.00     17155
   macro avg       1.00      1.00      1.00     17155
weighted avg       1.00      1.00      1.00     17155

weighted-avg prf: 0.9998251921815797, 0.9998251238705916, 0.9998251271114283


C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


Model Name:Decision Tree
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9478
           1       1.00      1.00      1.00      7677

    accuracy                           1.00     17155
   macro avg       1.00      1.00      1.00     17155
weighted avg       1.00      1.00      1.00     17155

weighted-avg prf: 0.9995919575142458, 0.9995919556980473, 0.9995919531716638


C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [7]:
df_report

,model,f1-score,precision,recall,support
0,BernoulliNB,0.826945,0.850198,0.804931,10668.000000
1,BernoulliNB,0.734603,0.705032,0.766764,6487.000000
accuracy,BernoulliNB,0.790498,0.790498,0.790498,0.790498
macro avg,BernoulliNB,0.780774,0.777615,0.785847,17155.000000
weighted avg,BernoulliNB,0.792027,0.795305,0.790498,17155.000000
